In [1]:
# after http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html
"""
Imports
"""
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import time
import os
import urllib.request
from tensorflow.models.rnn.ptb import reader

In [2]:
def gen_epochs(n, num_steps, batch_size):
    for i in range(n):
        yield reader.ptb_iterator(data, batch_size, num_steps)

def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


In [3]:
def train_network(g, num_epochs, num_steps = 120, batch_size = 32, verbose = True, checkpoint_file=None):
    tf.set_random_seed(2345)
    with tf.Session() as sess:
        init = tf.initialize_all_variables()
        sess.run(init)
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps, batch_size)):
            training_loss = 0
            steps = 0
            training_state = None
            for X, Y in epoch:
                steps += 1
                feed_dict={g['x']: X, g['y']: Y}
                if training_state is not None:
                    feed_dict[g['init_state']] = training_state
                training_loss_, training_state, _ = sess.run([g['total_loss'],
                                                    g['final_state'],
                                                    g['train_step']],
                                                    feed_dict)
                training_loss += training_loss_
            if verbose:
                print("Average training loss for Epoch", idx, ":", training_loss/steps)
            training_losses.append(training_loss/steps)
                    
        if checkpoint_file is not None:
            print("Saving variables to '%s'." % checkpoint_file)
            saver = tf.train.Saver()
            saver.save(sess, checkpoint_file)

    return training_losses

In [4]:
def build_graph(
    num_classes,
    cell_type = None,
    num_weights_for_custom_cell = 5,
    state_size = 256,
    batch_size = 32,
    num_steps = 120,
    num_layers = 3,
    build_with_dropout=False,
    learning_rate = 1e-4):

    reset_graph()

    x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
    y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')

    dropout = tf.constant(1.0)

    embeddings = tf.get_variable('embedding_matrix', [num_classes, state_size])

    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    if cell_type == 'Custom':
        cell = CustomCell(state_size, num_weights_for_custom_cell)
    elif cell_type == 'GRU':
        cell = tf.nn.rnn_cell.GRUCell(state_size)
    elif cell_type == 'LSTM':
        cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
    elif cell_type == 'LN_LSTM':
        cell = LayerNormalizedLSTMCell(state_size)
    else:
        cell = tf.nn.rnn_cell.BasicRNNCell(state_size)

    if build_with_dropout:
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=dropout)

    if cell_type == 'LSTM' or cell_type == 'LN_LSTM':
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
    else:
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)

    if build_with_dropout:
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout)

    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))

    #reshape rnn_outputs and y
    rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])
    y_reshaped = tf.reshape(y, [-1])

    logits = tf.matmul(rnn_outputs, W) + b

    predictions = tf.nn.softmax(logits)

    total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_reshaped))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

    return dict(
        x = x,
        y = y,
        init_state = init_state,
        final_state = final_state,
        total_loss = total_loss,
        train_step = train_step,
        preds = predictions
    )

In [5]:
def generate_characters(g, checkpoint_file, num_chars, prompt='A', pick_top_chars=None):
    """ Accepts a current character, initial state"""

    with tf.Session() as sess:
        
        sess.run(tf.initialize_all_variables())        
        saver = tf.train.Saver()
        print("Loading variables from '%s'.\n\n" % checkpoint_file)
        saver.restore(sess, checkpoint_file)
         

        state = None
        current_char = vocab_to_idx[prompt]
        chars = [current_char]

        for i in range(num_chars):
            if state is not None:
                feed_dict={g['x']: [[current_char]], g['init_state']: state}
            else:
                feed_dict={g['x']: [[current_char]]}

            preds, state = sess.run([g['preds'],g['final_state']], feed_dict)

            if pick_top_chars is not None:
                p = np.squeeze(preds)
                p[np.argsort(p)[:-pick_top_chars]] = 0
                p = p / np.sum(p)
                current_char = np.random.choice(vocab_size, 1, p=p)[0]
            else:
                current_char = np.random.choice(vocab_size, 1, p=np.squeeze(preds))[0]

            chars.append(current_char)

    chars = map(lambda x: idx_to_vocab[x], chars)
    print("".join(chars))
    return("".join(chars))



In [6]:
#file_name = '/home/key/Downloads/linux-4.9-rc7/kernel/all.c'
file_name = 'faust1.txt'
#file_name = 'racine.txt'
#file_name = 'mickiewicz.txt'

with open(file_name,'r') as f:
    raw_data = f.read()
    vocab = set(raw_data)
    
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))
vocab_size = len(vocab)

data = [vocab_to_idx[c] for c in raw_data]
print(raw_data[:100])
print(vocab)
print(idx_to_vocab)
del raw_data


  Zueignung.

  Ihr naht euch wieder, schwankende Gestalten,
  Die früh sich einst dem trüben Blick 
{'G', 'E', 'z', 'P', 'Z', 'O', 'r', 'l', 'ä', 'g', '?', 'Ü', 'J', 'L', 'm', 'U', 'y', 'a', 'Y', ';', '-', 'q', ' ', 'D', 'ß', 'ü', "'", 'B', 'F', 'Q', 'X', 'i', '"', 'M', 'x', 'W', 'C', 'h', 'V', 'o', 'c', 'b', 'w', 'T', 'k', 'N', 'p', 'A', ')', 'K', 'd', '!', ',', 'v', 'Ä', 'f', '(', '.', '\n', ':', 'H', 'I', 't', 's', 'R', 'j', 'n', 'Ö', 'u', 'ö', 'S', 'e'}
{0: 'G', 1: 'E', 2: 'z', 3: 'P', 4: 'Z', 5: 'O', 6: 'r', 7: 'l', 8: 'ä', 9: 'g', 10: '?', 11: 'Ü', 12: 'J', 13: 'L', 14: 'm', 15: 'U', 16: 'y', 17: 'a', 18: 'Y', 19: ';', 20: '-', 21: 'q', 22: ' ', 23: 'D', 24: 'ß', 25: 'ü', 26: "'", 27: 'B', 28: 'F', 29: 'Q', 30: 'X', 31: 'i', 32: '"', 33: 'M', 34: 'x', 35: 'W', 36: 'C', 37: 'h', 38: 'V', 39: 'o', 40: 'c', 41: 'b', 42: 'w', 43: 'T', 44: 'k', 45: 'N', 46: 'p', 47: 'A', 48: ')', 49: 'K', 50: 'd', 51: '!', 52: ',', 53: 'v', 54: 'Ä', 55: 'f', 56: '(', 57: '.', 58: '\n', 59: ':', 60: '

In [7]:
#save = 'models/linux_GRU_512_120'
checkpoint_file = 'models/faust_GRU_256_120'
#save = 'models/racine_GRU_256_120'
#save = 'models/mickiewicz_GRU_256_120'

state_size = 256
num_epochs = 40
train = True

In [8]:
if train:
    g1 = build_graph(num_classes=vocab_size, state_size = state_size, cell_type='GRU', num_steps=120)
    t = time.time()
    losses = train_network(g1, num_epochs=num_epochs, num_steps=120, checkpoint_file=checkpoint_file)
    print("It took", time.time() - t, "seconds to train the network.")
    print("The average loss on the final epoch was:", losses[-1])
    losses

Average training loss for Epoch 0 : 3.67351406696
Average training loss for Epoch 1 : 3.27894109838
Average training loss for Epoch 2 : 3.21779212297
Average training loss for Epoch 3 : 2.94823598862
Average training loss for Epoch 4 : 2.70722702438
Average training loss for Epoch 5 : 2.55438814444
Average training loss for Epoch 6 : 2.4260459554
Average training loss for Epoch 7 : 2.32698331627
Average training loss for Epoch 8 : 2.25762816971
Average training loss for Epoch 9 : 2.19821273579
Average training loss for Epoch 10 : 2.1454027447
Average training loss for Epoch 11 : 2.09950004839
Average training loss for Epoch 12 : 2.06024112655
Average training loss for Epoch 13 : 2.02631763383
Average training loss for Epoch 14 : 1.99621819047
Average training loss for Epoch 15 : 1.96872502682
Average training loss for Epoch 16 : 1.94318868132
Average training loss for Epoch 17 : 1.91924949721
Average training loss for Epoch 18 : 1.8967206221
Average training loss for Epoch 19 : 1.87550

In [9]:
g2 = build_graph(num_classes=vocab_size, cell_type='GRU', num_steps=1, batch_size=1)
prompt = 'C'
generate_characters(g2, checkpoint_file, 750, prompt=prompt)

Loading variables from 'models/faust_GRU_256_120'.


Cx,F,q H(ErYesen du werben,
  Ande Fluch vergreuen,
  Baß zang der Manze nur dößes.

  BELBER:
  Ich so ist'sdas nicht lein!  sü kchpie'n daschen flägen,
  Wenn es weure euch in der Linkost.


  ELOr einer schner;
  Dres Ermoch mir verkund nicht der Schänund bebören,
  Siaf alles hott ebagon,
  Düßtroß von einem Trofstreistarut.,
  Was Gand aber eahn,
  Der alö doren auch das Wenn!
  Zu hän bat von Fweibeud?

  Want.  aus)ejeusel auch.

  DIENDE (Wie Wein das warde but?
  Und du kerd sie gebagen!
  Daßehsorftern weicht Haus mich vor meinen Gracben.

  MEPHISTOPHELES:
  Gebst doch spfeilie vor.

  MIRHER:
  Gerauch verkolbstördes Glohche Prarz zu sipben-
  Nacht mefßes Verteug schönen Tard das wommest?
  Zust im wirqeine Kähre wie.

  FAUST:



''